In [1]:
import math
import pandas as pd
import numpy as np
from scipy import stats

In [86]:
np.random.seed(0)
a = np.random.randint(1, 100, 1000)
b = np.random.randint(1, 100, 1000)
c = np.random.randint(1, 100, 1000)
d = np.random.randint(1, 100, 1000)


## t-test

### 일표본 t-검정
- 그룹이 1

In [5]:
# 일표본 t-검정 시행
stats.ttest_1samp(a, a.mean())

Ttest_1sampResult(statistic=0.0, pvalue=1.0)

### 독립표본 t-검정 
 - 그룹이 2

In [69]:
data = pd.read_csv('C:/Users/user/Desktop/stats_data/Ch08.t-검정(t-test)2/Ch0801.IST.csv', encoding = 'utf-8-sig')
data.head()

,t_group,t_time
0,1,48187
1,1,47245
2,1,51020
3,1,50732
4,1,52416


In [70]:
# 분산 동질성 검사(등분산 검정)
stats.levene(data.t_time[data.t_group == 1], data.t_time[data.t_group == 2]) 

LeveneResult(statistic=0.43289078232812456, pvalue=0.5131769222686233)

- p값이 통상적인 기준인 0.05보다 크므로 두 그룹의 분산이 동일하다는 가정을 받아들임

In [7]:
# 독립표본 t-검정(등분산일 경우)
stats.ttest_ind(a, b, equal_var = True) # 이분산은 equal_var = False

Ttest_indResult(statistic=-0.9153810987612412, pvalue=0.3601020588446868)

- p값이 유의수준 0.05보다 크므로 귀무가설을 채택
- 즉, '두 데이터는 유의미한 차이가 없다'

### 대응표본 t-검정(paired)
- 한 사람을 두 번 측정

In [87]:
# 대응표본 t-검정 시행
stats.ttest_rel(a, b) # a-before, b-after

Ttest_relResult(statistic=-0.5040247223776022, pvalue=0.6143551380591987)

## 분산분석(ANOVA)

- 참고 : ANOVA 모형의 가정
    - 정규성: 독립변수에 따른 종속변수는 정규분포를 만족해야 함.
        - Shapiro-Wilk
    - 독립성: 독립변수 그룹군은 서로 독립적이어야 함.
        - 잔차검정
    - 분산의 동질성: 독립변수에 따른 종속변수 분포의 분산은 각 군마다 동일해야 함.
        - Hartley검정, Bartlett검정, Levene검정

### 일원 분산분석(One-Way ANAVA)
- 그룹이 3 이상

In [53]:
df = pd.read_csv('C:/Users/user/Desktop/stats_data/Ch09.분산분석(ANOVA)1/Ch0901.OWA.csv', encoding = 'utf-8-sig')
df.head(10)

,group,score
0,1,85
1,2,78
2,3,63
3,4,88
4,1,93
5,2,83
6,3,56
7,4,53
8,1,94
9,2,97


In [75]:
# 분산 동질성 검사(등분산 검정) - Levene
stats.levene(df.score[df.group == 1], df.score[df.group == 2], df.score[df.group == 3], df.score[df.group == 4]) 

LeveneResult(statistic=1.5622032060443554, pvalue=0.2014879807833761)

In [76]:
# 분산 동질성 검사(등분산 검정) - bartlett
stats.bartlett(df.score[df.group == 1], df.score[df.group == 2], df.score[df.group == 3], df.score[df.group == 4])

BartlettResult(statistic=6.0049371189812195, pvalue=0.11137027043376477)

- bartlett, levene 검정에서 모두 p-value가 0.05를 넘었으므로, 그룹들 간 분산이 동일하다는 귀무가설을 받아들임
- bartlett는 정규분포에 민감하기 때문에 levene를 많이 사용

In [77]:
# 분산이 동일하다는 가정하에 분산분석 실행
# 참고 - 이분산일 경우 비모수분석 실시

import statsmodels.formula.api as sm  # 회귀식 적합
from statsmodels.stats.anova import anova_lm  # 분산분석

# C(group)은 카테고리 타입 데이터임을 의미함. 필수로 넣어줘야 함
model = sm.ols(formula = 'score ~ C(group)', data = df)
model = model.fit()

In [78]:
anova_lm(model) 

,df,sum_sq,mean_sq,F,PR(>F)
C(group),3.0,919.115675,306.371892,3.54624,0.016346
Residual,135.0,11663.114541,86.393441,NaN,NaN


- 분산분석 결과, 매장별 서비스 점수의 차이는 차이가 있음(p < 0.05)

In [79]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd # 사후분석 - 튜키

posthoc = pairwise_tukeyhsd(df.score, df.group)
print(posthoc)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     1      2  -0.6809    0.9  -6.4825  5.1206  False
     1      3  -6.8684 0.0156  -12.774 -0.9628   True
     1      4  -2.8171 0.5405  -8.3288  2.6945  False
     2      3  -6.1875 0.0478 -12.3325 -0.0425   True
     2      4  -2.1362 0.7451  -7.9037  3.6312  False
     3      4   4.0513 0.2805  -1.8208  9.9234  False
-----------------------------------------------------


- 1, 3과 2, 3 데이터의 차이가 유의미함(p < 0.05)
- 그룹1(a)과 그룹2(b)
|~|지역1|지역2|지역3|지역4|
|--|--|--|--|--|
|Tukey|a|a, b|a, b| b|

### 반복측정 분산분석(Repeated Measures ANOVA)
- 그룹이 3이상, 반복측정

-------
-------